In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

import tensorflow as tf

import matplotlib.pyplot as plt

In [ ]:
IMG_SIZE    = 299
BATCH       = 64
EPOCHS_a    = 20
EPOCHS_b    = 10
SPLIT       = 0.3

In [ ]:
train_datagen = ImageDataGenerator(validation_split=SPLIT)
test_datagen = ImageDataGenerator()

train_it = train_datagen.flow_from_directory('data/train/', class_mode='categorical', batch_size=BATCH, target_size=(IMG_SIZE, IMG_SIZE), subset="training")
val_it = train_datagen.flow_from_directory('data/train/', class_mode='categorical', batch_size=BATCH, target_size=(IMG_SIZE, IMG_SIZE), subset="validation")
test_it = test_datagen.flow_from_directory('data/test/', class_mode='categorical', batch_size=BATCH)

In [ ]:
batchX, batchy = train_it.next()
NUM_CLASSES = batchy.shape[1]

In [ ]:
def build_model(num_classes, augment=False):
    x = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = InceptionResNetV2(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="Model")
    optimizer = tf.keras.optimizers.Adamax(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adamax(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model


In [ ]:
model_a = build_model(num_classes=NUM_CLASSES, augmentation=False)
hist = model_a.fit(train_it, epochs=EPOCHS_a, validation_data=val_it, verbose=1)

In [ ]:
plot_hist(hist)

In [ ]:
results_a = model_a.evaluate(test_it)
results_a

In [ ]:
model_b = unfreeze_model(model_a)
hist = model_b.fit(train_it, epochs=EPOCHS_b, validation_data=val_it, verbose=1)

In [ ]:
plot_hist(hist)

In [ ]:
results_b = model_b.evaluate(test_it)
results_b

In [ ]:
if results_a[1] > results_b[1]:
    model_a.save('model.h5')
else:
    model_b.save('model.h5')